In [1]:
# ============================================================
# CELL 1: IMPORTS & SETUP
# ============================================================

# --- PATH SETUP (MUST BE FIRST) ---
import sys
from pathlib import Path

# Get current directory (should be notebooks/)
current_dir = Path.cwd()
print(f"📁 Current directory: {current_dir}")
print(f"📁 Current directory name: {current_dir.name}")

# Go up one level to project root
project_root = current_dir.parent
print(f"📁 Project root: {project_root}")

# Verify database folder exists
database_path = project_root / 'database'
print(f"📁 Database path: {database_path}")
print(f"✅ Database folder exists: {database_path.exists()}")

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Added to Python path: {project_root}")
else:
    print(f"✅ Already in Python path")

print(f"\n📋 Python path (first 3):")
for i, p in enumerate(sys.path[:3], 1):
    print(f"   {i}. {p}")


# --- STANDARD LIBRARY IMPORTS ---
import os
import json
import time
import sqlite3
import glob
from typing import TypedDict, Optional, List, Dict, Any
from datetime import datetime
from enum import Enum

print("✅ Standard library imports loaded")


# --- ENVIRONMENT & CONFIGURATION ---
from dotenv import load_dotenv

# Load environment variables from project root
env_path = project_root / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path}")
else:
    print(f"⚠️  .env not found at: {env_path}")


# --- THIRD-PARTY ML/AI ---
import assemblyai as aai
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph
from pydantic import BaseModel, Field

print("✅ Third-party imports loaded")


# --- DATABASE IMPORTS ---
from database.db_operations import db
from database.models import Conversation, ConversationCreate

print("✅ Database imports loaded")
print("\n🎉 All imports loaded successfully!")

📁 Current directory: /home/manuel/Documents/tech/ai_content_ops/notebooks
📁 Current directory name: notebooks
📁 Project root: /home/manuel/Documents/tech/ai_content_ops
📁 Database path: /home/manuel/Documents/tech/ai_content_ops/database
✅ Database folder exists: True
✅ Added to Python path: /home/manuel/Documents/tech/ai_content_ops

📋 Python path (first 3):
   1. /home/manuel/Documents/tech/ai_content_ops
   2. /usr/lib/python313.zip
   3. /usr/lib/python3.13
✅ Standard library imports loaded
✅ Loaded .env from: /home/manuel/Documents/tech/ai_content_ops/.env
✅ Third-party imports loaded
✅ Database imports loaded

🎉 All imports loaded successfully!


In [2]:

# Cell 2 

# Test API key
assemblyai_key = os.getenv('ASSEMBLYAI_API_KEY')
print(f"AssemblyAI API Key loaded: {'✅' if assemblyai_key else '❌'}")
print(f"Key starts with: {assemblyai_key[:10] if assemblyai_key else 'None'}...")


AssemblyAI API Key loaded: ✅
Key starts with: 972365f41d...


In [3]:
# ============================================================
# CELL 3: LANGSMITH TRACING SETUP (OPTIONAL)
# ============================================================

import os
from dotenv import load_dotenv

# Load environment variables if not already loaded
load_dotenv()

# Get LangSmith API key from environment
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")

if langsmith_api_key:
    # Enable LangSmith tracing
    os.environ["LANGSMITH_API_KEY"] = langsmith_api_key
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "ai_content_ops"
    print("✅ LangSmith tracing enabled")
    print(f"   Project: ai_content_ops")
else:
    print("⚠️  LANGSMITH_API_KEY not found in .env")
    print("   LangSmith tracing disabled")
    print("   💡 Add LANGSMITH_API_KEY to your .env file to enable tracing")

✅ LangSmith tracing enabled
   Project: ai_content_ops


In [4]:
# Cell 3: Database Connection Test

conn = sqlite3.connect("data/app.db")
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("📊 Tables in app.db:")
for table in tables:
    table_name = table[0]
    
    # Get column info for each table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    print(f"\n🔧 {table_name}:")
    for col in columns:
        print(f"   - {col[1]} ({col[2]})")  # column_name (type)

conn.close()

📊 Tables in app.db:

🔧 conversations:
   - id (INTEGER)
   - title (TEXT)
   - raw_text (TEXT)
   - source (TEXT)
   - word_count (INTEGER)
   - created_at (DATETIME)
   - status (TEXT)

🔧 sqlite_sequence:
   - name ()
   - seq ()

🔧 blog_post_ideas:
   - id (INTEGER)
   - conversation_id (INTEGER)
   - title (TEXT)
   - description (TEXT)
   - usefulness_potential (INTEGER)
   - fitwith_seo_strategy (INTEGER)
   - fitwith_content_strategy (INTEGER)
   - inspiration_potential (INTEGER)
   - collaboration_potential (INTEGER)
   - innovation (INTEGER)
   - difficulty (INTEGER)
   - total_score (INTEGER)
   - sent_to_prod (BOOLEAN)
   - raw_llm_response (TEXT)
   - created_at (DATETIME)

🔧 processing_status:
   - id (INTEGER)
   - conversation_id (INTEGER)
   - stage (TEXT)
   - status (TEXT)
   - error_message (TEXT)
   - started_at (DATETIME)
   - completed_at (DATETIME)


In [5]:
## cell 4. Pydantic Model for Structured Output



class SpeakerRole(str, Enum):
    """Possible speaker roles in the conversation"""
    CLIENT = "client"
    INTERVIEWER = "interviewer"

class Speaker(BaseModel):
    """Information about a person speaking in the conversation"""
    name: Optional[str] = Field(default=None, description="Name of the speaker if mentioned")
    role: Optional[SpeakerRole] = Field(default=None, description="Role of the speaker in the conversation")
    company: Optional[str] = Field(default=None, description="Company they work for if mentioned")

class Challenge(BaseModel):
    """A challenge or problem mentioned in the conversation"""
    description: Optional[str] = Field(default=None, description="Description of the challenge")
    impact: Optional[str] = Field(default=None, description="How this challenge affects them")
    urgency: Optional[str] = Field(default=None, description="Low, Medium, or High urgency")

class CurrentSolution(BaseModel):
    """How they currently solve their problems"""
    solution: Optional[str] = Field(default=None, description="What they're currently doing")
    satisfaction_level: Optional[str] = Field(default=None, description="How satisfied they are: Very Satisfied, Satisfied, Neutral, Unsatisfied, Very Unsatisfied")
    limitations: Optional[List[str]] = Field(default=[], description="Limitations of current solution")

class Need(BaseModel):
    """A need identified using psychology frameworks like NVC"""
    need_category: Optional[str] = Field(default=None, description="Category of need (e.g., autonomy, efficiency, security, connection)")
    description: Optional[str] = Field(default=None, description="Specific need description")
    intensity: Optional[str] = Field(default=None, description="Low, Medium, or High intensity")

class ExtractedInsights(BaseModel):
    """Complete structured output from conversation analysis"""
    
    # Speakers
    speakers: Optional[List[Speaker]] = Field(default=[], description="People identified in the conversation")
    
    # What they care about
    core_values: Optional[List[str]] = Field(default=[], description="What this person/company cares about most")
    priorities: Optional[List[str]] = Field(default=[], description="Their current priorities and focus areas")
    
    # Challenges
    primary_challenges: Optional[List[Challenge]] = Field(default=[], description="Main problems they're facing")
    secondary_challenges: Optional[List[Challenge]] = Field(default=[], description="Secondary or related problems")
    
    # Current solutions
    current_solutions: Optional[List[CurrentSolution]] = Field(default=[], description="How they solve problems today")
    
    # Needs analysis
    psychological_needs: Optional[List[Need]] = Field(default=[], description="Underlying needs using NVC or similar frameworks")
 


In [6]:
# Cell 5: Raw Blog Idea Model (Simple)
class RawBlogIdea(BaseModel):
    """Raw blog idea from creative agent"""
    title: str
    description: str
    target_audience: str
    content_angle: str
    business_value: str

print("✅ Simple RawBlogIdea model ready")

✅ Simple RawBlogIdea model ready


In [7]:
def validate_raw_blog_ideas(raw_ideas: List[Dict]) -> List[RawBlogIdea]:
    """Validate and convert raw JSON to Pydantic models"""
    validated_ideas = []
    
    for idea in raw_ideas:
        try:
            validated_idea = RawBlogIdea(**idea)
            validated_ideas.append(validated_idea)
        except Exception as e:
            print(f"⚠️ Invalid blog idea skipped: {e}")
    
    print(f"✅ Validated {len(validated_ideas)} out of {len(raw_ideas)} raw ideas")
    return validated_ideas

print("✅ RawBlogIdea model and validation ready")

✅ RawBlogIdea model and validation ready


In [8]:
# Cell 6: Update AudioPipelineState for 5-Node Pipeline
class AudioPipelineState(TypedDict):
    # File info
    file_path: str
    filename: str
    
    # Processing results
    transcript_text: Optional[str]
    conversation_id: Optional[int]
    extracted_insights: Optional[ExtractedInsights]
    raw_blog_ideas: Optional[List[Dict]]        # From creative agent (Node 4)
    scored_blog_ideas: Optional[List[Dict]]     # From analyst agent (Node 5) 
    saved_idea_ids: Optional[List[int]]         # From database saver (Node 6) 
    
    
    # Status & error handling
    status: str
    error: Optional[str]

print("✅ Updated AudioPipelineState for 6-node pipeline")

✅ Updated AudioPipelineState for 6-node pipeline


In [9]:
# Cell: Individual Strategy Loader Function

def load_company_strategy():
    """Load company strategy document"""
    try:
        company_strategy_path = "../data/processed/company_strategy.mkd"
        if os.path.exists(company_strategy_path):
            with open(company_strategy_path, "r", encoding="utf-8") as f:
                content = f.read()
            print(f"✅ Loaded company strategy ({len(content)} chars)")
            return content
        else:
            print("⚠️ Company strategy document not found")
            return "Company strategy document not available."
    except Exception as e:
        print(f"❌ Error loading company strategy: {e}")
        return "Company strategy document not available."

def load_seo_strategy():
    """Load SEO strategy document"""
    try:
        seo_strategy_path = "../data/processed/seo_strategy.mkd"
        if os.path.exists(seo_strategy_path):
            with open(seo_strategy_path, "r", encoding="utf-8") as f:
                content = f.read()
            print(f"✅ Loaded SEO strategy ({len(content)} chars)")
            return content
        else:
            print("⚠️ SEO strategy document not found")
            return "SEO strategy document not available."
    except Exception as e:
        print(f"❌ Error loading SEO strategy: {e}")
        return "SEO strategy document not available."

def load_content_strategy():
    """Load content strategy document"""
    try:
        content_strategy_path = "../data/processed/content_strategy.mkd"
        if os.path.exists(content_strategy_path):
            with open(content_strategy_path, "r", encoding="utf-8") as f:
                content = f.read()
            print(f"✅ Loaded content strategy ({len(content)} chars)")
            return content
        else:
            print("⚠️ Content strategy document not found")
            return "Content strategy document not available."
    except Exception as e:
        print(f"❌ Error loading content strategy: {e}")
        return "Content strategy document not available."

def prepare_strategy_context_for_scoring():
    """Prepare full strategy context for scoring (used by analyst agent)"""
    return {
        'company_strategy_summary': load_company_strategy(),
        'seo_strategy_summary': load_seo_strategy(),
        'content_strategy_summary': load_content_strategy()
    }

print("✅ Individual strategy loader functions ready")
print("✅ prepare_strategy_context_for_scoring() ready")

✅ Individual strategy loader functions ready
✅ prepare_strategy_context_for_scoring() ready


In [10]:
# Cell 7: Updated Company Strategy Context Loader (3 Documents)
def load_company_strategy_context():
    """Load company strategy, SEO strategy, and content strategy for context"""
    
    strategy_context = {}
    
    try:
        # Load company strategy
        company_strategy_path = "../data/processed/company_strategy.mkd"
        if os.path.exists(company_strategy_path):
            with open(company_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["company_strategy"] = f.read()
            print(f"✅ Loaded company strategy ({len(strategy_context['company_strategy'])} chars)")
        else:
            strategy_context["company_strategy"] = "Company strategy document not available."
            print("⚠️ Company strategy document not found")
        
        # Load SEO strategy
        seo_strategy_path = "../data/processed/seo_strategy.mkd"
        if os.path.exists(seo_strategy_path):
            with open(seo_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["seo_strategy"] = f.read()
            print(f"✅ Loaded SEO strategy ({len(strategy_context['seo_strategy'])} chars)")
        else:
            strategy_context["seo_strategy"] = "SEO strategy document not available."
            print("⚠️ SEO strategy document not found")
        
        # Load content strategy (NEW)
        content_strategy_path = "../data/processed/content_strategy.mkd"
        if os.path.exists(content_strategy_path):
            with open(content_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["content_strategy"] = f.read()
            print(f"✅ Loaded content strategy ({len(strategy_context['content_strategy'])} chars)")
        else:
            strategy_context["content_strategy"] = "Content strategy document not available."
            print("⚠️ Content strategy document not found")
            
    except Exception as e:
        print(f"❌ Error loading strategy documents: {e}")
        strategy_context = {
            "company_strategy": "Strategy document not available",
            "seo_strategy": "SEO strategy document not available", 
            "content_strategy": "Content strategy document not available"
        }
    
    return strategy_context

# Test loading all three documents
strategy_context = load_company_strategy_context()
print(f"📊 Strategy context keys: {list(strategy_context.keys())}")
print(f"📊 Total context size: {sum(len(v) for v in strategy_context.values() if isinstance(v, str))} chars")

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
📊 Strategy context keys: ['company_strategy', 'seo_strategy', 'content_strategy']
📊 Total context size: 12144 chars


In [11]:
# Batch Processing Function (Updated with Full Insights Display)
def process_audio_batch(audio_files: List[Path], pipeline) -> dict:
    """Process all audio files in batch with detailed insights display"""
    
    if not audio_files:
        print("❌ No files to process")
        return {"processed": [], "failed": [], "total": 0}
    
    print(f"\n🚀 STARTING BATCH PROCESSING - {len(audio_files)} files")
    print("=" * 60)
    
    processed_files = []
    failed_files = []
    results = []
    
    for i, file_path in enumerate(audio_files, 1):
        print(f"\n📂 Processing {i}/{len(audio_files)}: {file_path.name}")
        print("-" * 40)
        
        # Create initial state
        initial_state = {
            "file_path": str(file_path),
            "filename": file_path.name,
            "transcript_text": None,
            "conversation_id": None,
            "extracted_insights": None,  
            "error": None,
            "status": "processing"
        }
        
        try:
            # Run through pipeline
            result = pipeline.invoke(initial_state)
            
            if result["status"] in ["completed", "insights_extracted"]:
                print(f"✅ SUCCESS: {file_path.name}")
                print(f"   Conversation ID: {result['conversation_id']}")
                print(f"   Transcript preview: {result['transcript_text'][:100]}...")
                
                # FULL INSIGHTS DISPLAY
                if result.get('extracted_insights'):
                    insights = result['extracted_insights']
                    print(f"\n🧠 === EXTRACTED INSIGHTS FOR: {file_path.name} ===")
                    print("=" * 50)
                    
                    # Speakers
                    if insights.speakers:
                        print("👥 SPEAKERS:")
                        for speaker in insights.speakers:
                            print(f"   • Name: {speaker.name or 'Unknown'}")
                            print(f"     Role: {speaker.role or 'Unknown'}")  
                            print(f"     Company: {speaker.company or 'Unknown'}")
                    
                    # Core Values
                    if insights.core_values:
                        print("💎 CORE VALUES:")
                        for value in insights.core_values:
                            print(f"   • {value}")
                    
                    # Priorities
                    if insights.priorities:
                        print("🎯 PRIORITIES:")
                        for priority in insights.priorities:
                            print(f"   • {priority}")
                    
                    # Primary Challenges
                    if insights.primary_challenges:
                        print("🔥 PRIMARY CHALLENGES:")
                        for challenge in insights.primary_challenges:
                            print(f"   • Challenge: {challenge.description}")
                            print(f"     Impact: {challenge.impact}")
                            print(f"     Urgency: {challenge.urgency}")
                    
                    # Secondary Challenges
                    if insights.secondary_challenges:
                        print("⚠️  SECONDARY CHALLENGES:")
                        for challenge in insights.secondary_challenges:
                            print(f"   • Challenge: {challenge.description}")
                            print(f"     Impact: {challenge.impact}")
                            print(f"     Urgency: {challenge.urgency}")
                    
                    # Current Solutions
                    if insights.current_solutions:
                        print("🔧 CURRENT SOLUTIONS:")
                        for solution in insights.current_solutions:
                            print(f"   • Solution: {solution.solution}")
                            print(f"     Satisfaction: {solution.satisfaction_level}")
                            if solution.limitations:
                                print(f"     Limitations: {', '.join(solution.limitations)}")
                    
                    # Psychological Needs
                    if insights.psychological_needs:
                        print("🧘 PSYCHOLOGICAL NEEDS:")
                        for need in insights.psychological_needs:
                            print(f"   • {need.description}")
                            print(f"     Category: {need.need_category}")
                            print(f"     Intensity: {need.intensity}")
                    
                    print("🧠 === END INSIGHTS ===")
                    print("-" * 50)
                
                processed_files.append(file_path)
            else:
                print(f"❌ FAILED: {file_path.name}")
                print(f"   Status: {result.get('status', 'Unknown')}")
                print(f"   Error: {result.get('error', 'Unknown error')}")
                failed_files.append(file_path)
            
            results.append(result)
            
        except Exception as e:
            print(f"❌ PIPELINE ERROR: {file_path.name}")
            print(f"   Exception: {str(e)}")
            failed_files.append(file_path)
            
            results.append({
                **initial_state,
                "error": str(e),
                "status": "pipeline_error"
            })
    
    # Final Summary
    print(f"\n📊 BATCH PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"✅ Successfully processed: {len(processed_files)}")
    print(f"❌ Failed: {len(failed_files)}")
    print(f"📁 Total files: {len(audio_files)}")
    
    if failed_files:
        print(f"\n❌ Failed files:")
        for failed_file in failed_files:
            print(f"   - {failed_file.name}")
    
    return {
        "processed": processed_files,
        "failed": failed_files,
        "total": len(audio_files),
        "results": results
    }

print("✅ Batch processing function ready with full insights display")

✅ Batch processing function ready with full insights display


In [12]:
# Cell 5: Batch File Discovery and Management


def find_audio_files_in_temp(temp_folder: Path = None) -> List[Path]:
    """Find all audio files in temp folder"""
    
    # Use default temp folder if not specified
    if temp_folder is None:
        temp_folder = project_root / 'data' / 'temp'
    
    # Ensure folder exists
    temp_folder.mkdir(parents=True, exist_ok=True)
    
    # Check if folder exists
    if not temp_folder.exists():
        print(f"❌ Temp folder not found: {temp_folder}")
        return []
    
    # Find audio files
    audio_extensions = ['*.wav', '*.mp3', '*.m4a']
    audio_files = []
    
    for ext in audio_extensions:
        files = list(temp_folder.glob(ext))
        audio_files.extend(files)
    
    return sorted(audio_files)


def display_batch_info(audio_files: List[Path]) -> bool:
    """Display information about the batch of files"""
    
    if not audio_files:
        print("❌ No audio files found in temp folder!")
        print("💡 TIP: Add .wav files to data/temp/ folder")
        return False
    
    total_size_mb = sum(f.stat().st_size for f in audio_files) / (1024 * 1024)
    
    print(f"📊 BATCH INFO:")
    print(f"   Files to process: {len(audio_files)}")
    print(f"   Total size: {total_size_mb:.1f} MB")
    print(f"\n📁 Files found:")
    
    for i, file_path in enumerate(audio_files, 1):
        size_mb = file_path.stat().st_size / (1024 * 1024)
        print(f"   {i}. {file_path.name} ({size_mb:.1f} MB)")
    
    return True


print("✅ File management functions defined")

✅ File management functions defined


In [13]:
# Cell: Extract Insights Function - ROLE FIXED VERSION
def extract_insights_from_transcript(transcript: str) -> ExtractedInsights:
    """Extract structured insights using Anthropic Claude - ROLE FIXED VERSION"""
    
    prompt = f"""
    Analyze this conversation transcript and extract structured insights:
    
    Transcript: {transcript}
    
    IMPORTANT: For speaker roles, use ONLY these exact values:
    - "client" for the person being interviewed/consulted (CTO, CEO, Manager, business owner, etc.)
    - "interviewer" for the person asking questions or conducting the interview
    
    Extract the following information in JSON format:
    - speakers: List of people mentioned with name, role (client/interviewer only), company
    - core_values: What they care about most  
    - priorities: Current focus areas
    - primary_challenges: Main problems they face with description, impact, urgency
    - secondary_challenges: Secondary problems
    - current_solutions: How they solve problems now with satisfaction level
    - psychological_needs: Underlying needs with category, description, intensity
    
    Return ONLY valid JSON in this exact structure - no markdown, no code blocks:
    {{
        "speakers": [
            {{"name": "Manuel", "role": "client", "company": "Drone flytech"}}
        ],
        "core_values": ["efficiency", "transparency"],
        "priorities": ["improving processes"],
        "primary_challenges": [
            {{
                "description": "Tracking payment issues",
                "impact": "Creates confusion in processes", 
                "urgency": "High"
            }}
        ],
        "secondary_challenges": [
            {{
                "description": "Secondary challenge",
                "impact": "Secondary impact",
                "urgency": "Medium"
            }}
        ],
        "current_solutions": [
            {{
                "solution": "Current approach",
                "satisfaction_level": "Neutral",
                "limitations": ["limitation1", "limitation2"]
            }}
        ],
        "psychological_needs": [
            {{
                "need_category": "security",
                "description": "Need for confidence",
                "intensity": "High"
            }}
        ]
    }}
    
    Remember: 
    - Use "client" for Manuel (even though he's CTO)
    - Use "interviewer" for the person asking questions
    - Use exact urgency values: "Low", "Medium", "High"
    - Use exact satisfaction levels: "Very Satisfied", "Satisfied", "Neutral", "Unsatisfied", "Very Unsatisfied"
    - Use exact intensity values: "Low", "Medium", "High"
    """
    
    try:
        # Use the Claude LLM you already set up
        response = llm.invoke(prompt)
        
        print(f"📝 Raw response length: {len(response.content)} chars")
        print(f"📝 Response starts with: {response.content[:50]}...")
        
        # Clean markdown code blocks
        content = response.content.strip()
        if content.startswith('```json'):
            print("🔧 Removing JSON markdown blocks...")
            content = content.replace('```json', '').replace('```', '').strip()
            print(f"🔧 Cleaned content starts with: {content[:50]}...")
        
        # Parse the cleaned JSON response
        insights_data = json.loads(content)
        
        # Convert to Pydantic model
        result = ExtractedInsights(**insights_data)
        print(f"✅ Successfully extracted insights with correct speaker roles!")
        return result
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        print(f"📝 Raw response: {response.content[:500]}...")
        raise
    except Exception as e:
        print(f"❌ Error in LLM call: {e}")
        raise

print("✅ Updated extract_insights_from_transcript with speaker role fix")

✅ Updated extract_insights_from_transcript with speaker role fix


In [14]:
# Cell: Fixed Creative Agent Function
def generate_blog_ideas_from_insights(insights: ExtractedInsights, strategy_context: dict) -> List[Dict]:
    """
    Fixed creative agent that handles Claude's markdown JSON response
    """
    
    creative_prompt = f"""
    You are a creative content strategist for Big Kids Automation, a company that helps businesses implement AI and automation solutions.
    
    COMPANY CONTEXT:
    {strategy_context.get('company_strategy', 'Strategy not available')[:1000]}...
    
    SEO STRATEGY:
    {strategy_context.get('seo_strategy', 'SEO strategy not available')[:500]}...
    
    CONVERSATION INSIGHTS TO WORK FROM:
    
    Speakers: {[f"{s.name} ({s.role}) from {s.company}" for s in insights.speakers] if insights.speakers else "Unknown speakers"}
    
    Core Values: {", ".join(insights.core_values) if insights.core_values else "None identified"}
    
    Priorities: {", ".join(insights.priorities) if insights.priorities else "None identified"}
    
    Primary Challenges:
    {chr(10).join([f"- {c.description} (Impact: {c.impact}, Urgency: {c.urgency})" for c in insights.primary_challenges]) if insights.primary_challenges else "None identified"}
    
    Current Solutions:
    {chr(10).join([f"- {s.solution} (Satisfaction: {s.satisfaction_level})" for s in insights.current_solutions]) if insights.current_solutions else "None identified"}
    
    Psychological Needs:
    {chr(10).join([f"- {n.description} ({n.need_category}, {n.intensity} intensity)" for n in insights.psychological_needs]) if insights.psychological_needs else "None identified"}
    
    TASK:
    Generate 4-5 creative blog post ideas that:
    1. Address the challenges and needs identified in this conversation
    2. Align with Big Kids Automation's mission to help businesses with AI/automation
    3. Provide value to potential clients facing similar challenges
    4. Support our SEO and content marketing strategy
    5. Are actionable and practical, not just theoretical
    
    For each blog post idea, provide:
    - title: Clear, engaging title that includes relevant keywords
    - description: 2-3 sentence description of what the post will cover
    - target_audience: Who this post is primarily for
    - content_angle: The unique angle or approach this post takes
    - business_value: How this post helps our business goals
    
    IMPORTANT: Return ONLY the JSON array, no markdown formatting, no code blocks, no explanatory text.
    
    Format:
    [
        {{
            "title": "How AI Proposal Systems Balance Speed with Brand Differentiation",
            "description": "A practical guide showing how modern AI-powered proposal systems solve the common problem of maintaining company uniqueness while leveraging automation. Includes real case studies and implementation steps.",
            "target_audience": "Business development directors and proposal managers at consulting firms",
            "content_angle": "Problem-solution with real case studies",
            "business_value": "Attracts prospects struggling with proposal automation while maintaining differentiation"
        }}
    ]
    """
    
    try:
        # Generate ideas using Claude
        response = llm.invoke(creative_prompt)
        raw_content = response.content.strip()
        
        print(f"📝 Raw response length: {len(raw_content)} chars")
        print(f"📝 Response starts with: {raw_content[:50]}...")
        
        # Handle markdown code blocks
        if raw_content.startswith('```'):
            print("🔧 Removing markdown code blocks...")
            # Remove ```json and ``` wrappers
            lines = raw_content.split('\n')
            # Remove first line if it's ```json or ```
            if lines[0].startswith('```'):
                lines = lines[1:]
            # Remove last line if it's ```
            if lines and lines[-1].strip() == '```':
                lines = lines[:-1]
            raw_content = '\n'.join(lines).strip()
            print(f"🔧 Cleaned content starts with: {raw_content[:50]}...")
        
        # Parse JSON response
        blog_ideas = json.loads(raw_content)
        
        print(f"✅ Creative agent successfully parsed {len(blog_ideas)} blog ideas")
        return blog_ideas
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error in creative agent: {e}")
        print(f"📝 Cleaned content: {raw_content[:500]}...")
        return []
    except Exception as e:
        print(f"❌ Error in creative agent: {e}")
        return []

print("✅ Fixed creative agent function ready")

✅ Fixed creative agent function ready


In [15]:
# Cell 17: Updated Scoring Engine with Content Strategy Context
def score_blog_idea_with_llm(idea: dict, strategy_context: dict, conversation_context: str = "") -> dict:
    """Score a single blog idea using LLM with all three strategy contexts"""
    
    scoring_prompt = f"""
    You are an expert content strategist for Big Kids Automation. Score this blog post idea on a 1-10 scale using our strategic context.
    
    COMPANY STRATEGY:
    {strategy_context.get('company_strategy_summary', 'Not available')}
    
    SEO STRATEGY:
    {strategy_context.get('seo_strategy_summary', 'Not available')}
    
    CONTENT STRATEGY:
    {strategy_context.get('content_strategy_summary', 'Not available')}
    
    BLOG IDEA TO SCORE:
    Title: {idea.get('title', 'No title')}
    Description: {idea.get('description', 'No description')}
    Target Audience: {idea.get('target_audience', 'Unknown')}
    Business Value: {idea.get('business_value', 'Unknown')}
    Content Angle: {idea.get('content_angle', 'Unknown')}
    
    CONVERSATION CONTEXT:
    {conversation_context[:300] if conversation_context else 'No context available'}...
    
    SCORING INSTRUCTIONS:
    Rate each criterion from 1-10 (10 = excellent, 1 = poor):
    
    1. usefulness_potential: How useful will this be to readers with real problems?
    2. fitwith_seo_strategy: How well does this align with our SEO keywords and strategy?
    3. fitwith_content_strategy: How well does this fit our content strategy, voice, and approach?
    4. inspiration_potential: How likely to inspire readers to take meaningful action?
    5. collaboration_potential: How likely to generate leads/prospects who contact us?
    6. innovation: How unique is this topic compared to existing content?
    7. difficulty: How complex/time-consuming will this be to write? (1=very hard, 10=easy)
    
    Return ONLY valid JSON with your scores and brief reasoning:
    {{
        "usefulness_potential": 8,
        "fitwith_seo_strategy": 7,
        "fitwith_content_strategy": 9,
        "inspiration_potential": 6,
        "collaboration_potential": 8,
        "innovation": 7,
        "difficulty": 4,
        "reasoning": "This idea scores well because it aligns with our content strategy focus on..."
    }}
    """
    
    # ... rest of the function stays the same
    try:
        response = llm.invoke(scoring_prompt)
        
        content = response.content.strip()
        if content.startswith('```json'):
            content = content.replace('```json', '').replace('```', '').strip()
        
        scores = json.loads(content)
        
        # Validate scores are in range
        for criterion in ['usefulness_potential', 'fitwith_seo_strategy', 'fitwith_content_strategy', 
                         'inspiration_potential', 'collaboration_potential', 'innovation', 'difficulty']:
            if criterion in scores:
                scores[criterion] = max(1, min(10, scores[criterion]))
        
        # Calculate total score
        total_score = sum([
            scores.get('usefulness_potential', 5),
            scores.get('fitwith_seo_strategy', 5),
            scores.get('fitwith_content_strategy', 5),
            scores.get('inspiration_potential', 5),
            scores.get('collaboration_potential', 5),
            scores.get('innovation', 5),
            scores.get('difficulty', 5)
        ])
        
        scores['total_score'] = total_score
        return scores
        
    except Exception as e:
        print(f"❌ Error scoring idea: {e}")
        return {
            "usefulness_potential": 5, "fitwith_seo_strategy": 5, "fitwith_content_strategy": 5,
            "inspiration_potential": 5, "collaboration_potential": 5, "innovation": 5,
            "difficulty": 5, "total_score": 35, "reasoning": f"Default scores due to error: {str(e)}"
        }

print("✅ Updated LLM scoring engine with content strategy context")

✅ Updated LLM scoring engine with content strategy context


In [16]:
# Cell 6: Define LangGraph Nodes
def transcription_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 1: Transcribe audio file with AssemblyAI"""
    try:
        print(f"🎙️ Transcribing: {state['filename']}")
        
        # Configure transcriber
        transcriber = aai.Transcriber()
        
        # Transcribe the file
        transcript = transcriber.transcribe(state['file_path'])
        
        if transcript.status == aai.TranscriptStatus.error:
            return {
                **state,
                "error": f"AssemblyAI error: {transcript.error}",
                "status": "transcription_failed"
            }
        
        return {
            **state,
            "transcript_text": transcript.text,
            "status": "transcribed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Transcription error: {str(e)}",
            "status": "transcription_failed"
        }

def database_saver_node_conversations(state: AudioPipelineState) -> AudioPipelineState:
    """Node 2: Save transcript to database"""
    try:
        print(f"💾 Saving to database: {state['filename']}")
        
        # Create conversation object
        conversation = ConversationCreate(
            title=f"Audio: {state['filename']}",
            raw_text=state['transcript_text'],
            source="transcribed"
        )
        
        # Save to database
        conversation_id = db.create_conversation(conversation)
        
        return {
            **state,
            "conversation_id": conversation_id,
            "status": "completed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Database error: {str(e)}",
            "status": "database_failed"
        }

print("✅ LangGraph nodes defined")

✅ LangGraph nodes defined


In [17]:
def pain_extractor_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node: Extract structured insights from conversation transcript
    """
    print("🧠 Starting pain extraction...")
    
    try:
        # Extract insights using OpenAI structured output
        insights = extract_insights_from_transcript(state['transcript_text'])
        
        if insights:
            print(f"✅ Extracted insights: {len(insights.primary_challenges)} primary challenges, {len(insights.speakers)} speakers")
            
            return {
                **state,
                "extracted_insights": insights,
                "status": "insights_extracted"
            }
        else:
            return {
                **state,
                "error": "Failed to extract insights from transcript",
                "status": "error"
            }
            
    except Exception as e:
        print(f"❌ Pain extraction failed: {e}")
        return {
            **state,
            "error": f"Pain extraction error: {str(e)}",
            "status": "error"
        }

In [18]:
# Cell: Creative Agent Node - FORCE RELOAD
def creative_agent_node(state: AudioPipelineState) -> AudioPipelineState:
    """Creative agent that generates raw blog ideas"""
    
    try:
        print("🎨 Starting creative blog idea generation...")
        
        insights = state.get('extracted_insights')
        if not insights:
            return {**state, "error": "No insights available", "status": "error"}
        
        print(f"📊 Working with insights: {len(insights.primary_challenges)} challenges")
        
        # Load strategy context
        strategy_context = load_company_strategy_context()
        
        # Generate ideas (returns JSON list)
        raw_ideas_json = generate_blog_ideas_from_insights(insights, strategy_context)
        
        if not raw_ideas_json:
            return {**state, "error": "No ideas generated", "status": "error"}
        
        # Convert to Pydantic for validation
        validated_ideas = []
        for idea_json in raw_ideas_json:
            try:
                idea = RawBlogIdea(**idea_json)
                validated_ideas.append(idea)
            except Exception as e:
                print(f"⚠️ Skipping invalid idea: {e}")
        
        if validated_ideas:
            print(f"🎉 Generated {len(validated_ideas)} valid blog ideas")
            
            # Convert back to dict for state storage
            ideas_as_dicts = [idea.model_dump() for idea in validated_ideas]
            
            return {
                **state,
                "raw_blog_ideas": ideas_as_dicts,
                "status": "ideas_generated"
            }
        else:
            return {**state, "error": "No valid ideas after validation", "status": "error"}
            
    except Exception as e:
        print(f"❌ Creative agent error: {e}")
        import traceback
        traceback.print_exc()
        return {**state, "error": str(e), "status": "error"}

print("✅ Creative agent node RELOADED")

✅ Creative agent node RELOADED


In [19]:
# Cell 19: Analyst Agent Node - FIXED for Pydantic Objects
def analyst_agent_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node that scores blog ideas using company strategy context
    Input: state["raw_blog_ideas"] 
    Output: state["scored_blog_ideas"]
    """
    
    try:
        print("🔍 Starting analyst agent - scoring blog ideas...")
        
        # Check current status
        current_status = state.get('status', '')
        print(f"📊 Input status: {current_status}")
        
        # Check if we have raw blog ideas to score
        raw_ideas = state.get('raw_blog_ideas')
        if not raw_ideas:
            return {
                **state,
                "error": "No raw blog ideas available for scoring",
                "status": "error"
            }
        
        print(f"📊 Found {len(raw_ideas)} blog ideas to score")
        
        # Load strategy context for scoring
        print("📚 Loading strategy context...")
        strategy_context = prepare_strategy_context_for_scoring()
        
        # Get conversation context for better scoring
        conversation_context = state.get('transcript_text', '')
        
        # Score each blog idea
        scored_ideas = []
        for i, idea in enumerate(raw_ideas, 1):
            # FIXED: Handle both Pydantic objects and dicts properly
            if hasattr(idea, 'title'):
                # It's a Pydantic object - convert to dict first
                idea_dict = idea.model_dump() if hasattr(idea, 'model_dump') else idea.__dict__
                title_preview = idea.title[:50]
            else:
                # It's already a dict
                idea_dict = idea
                title_preview = idea.get('title', 'No title')[:50]
            
            print(f"🔍 Scoring idea {i}/{len(raw_ideas)}: {title_preview}...")
            
            # Score the idea (now always working with dict)
            scores = score_blog_idea_with_llm(idea_dict, strategy_context, conversation_context)
            
            # Combine original idea with scores
            scored_idea = {
                **idea_dict,  # Original idea data (now definitely a dict)
                **scores      # Scoring data
            }
            
            scored_ideas.append(scored_idea)
            
            print(f"   ✅ Scored: {scores.get('total_score', 0)}/70 points")
        
        # Sort by total score (highest first)
        scored_ideas.sort(key=lambda x: x.get('total_score', 0), reverse=True)
        
        print(f"\n🎉 Analyst agent completed scoring!")
        print(f"📊 Scored {len(scored_ideas)} ideas")
        
        if scored_ideas:
            print(f"🏆 Top idea: '{scored_ideas[0].get('title', 'Unknown')[:50]}...' ({scored_ideas[0].get('total_score', 0)}/70)")
            print(f"📉 Lowest idea: '{scored_ideas[-1].get('title', 'Unknown')[:50]}...' ({scored_ideas[-1].get('total_score', 0)}/70)")
        
        return {
            **state,
            "scored_blog_ideas": scored_ideas,
            "status": "ideas_scored"
        }
        
    except Exception as e:
        print(f"❌ Error in analyst agent node: {e}")
        import traceback
        traceback.print_exc()
        
        return {
            **state,
            "error": f"Analyst agent error: {str(e)}",
            "status": "error"
        }

print("✅ Analyst agent node FIXED for Pydantic objects")

✅ Analyst agent node FIXED for Pydantic objects


In [20]:
# Cell 4: Test AssemblyAI Connection
# Configure AssemblyAI
aai.settings.api_key = os.getenv('ASSEMBLYAI_API_KEY')

# Test with a simple transcription (we'll use a file from temp folder)
def test_assemblyai_connection():
    """Test if AssemblyAI is working"""
    try:
        # Just test the API key is valid
        transcriber = aai.Transcriber()
        print("✅ AssemblyAI connection successful")
        return True
    except Exception as e:
        print(f"❌ AssemblyAI connection failed: {e}")
        return False

test_assemblyai_connection()

✅ AssemblyAI connection successful


True

In [21]:
# Cell: Database Saver Node for BLOG IDEAS (Node 6) - FIXED RETURN
from database.models import BlogPostIdeaCreate

def database_saver_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node that saves scored blog ideas to database
    Input: state["scored_blog_ideas"]
    Output: state["saved_idea_ids"]
    """
    
    try:
        print("💾 Starting database saver - saving scored blog ideas...")
        
        scored_ideas = state.get('scored_blog_ideas')
        conversation_id = state.get('conversation_id')
        
        if not scored_ideas:
            print("❌ No scored blog ideas available to save")
            return {
                **state,
                "error": "No scored blog ideas available to save",
                "status": "error"
            }
        
        if not conversation_id:
            print("❌ No conversation_id available for linking ideas")
            return {
                **state,
                "error": "No conversation_id available for linking ideas",
                "status": "error"
            }
        
        print(f"📊 Found {len(scored_ideas)} scored ideas to save")
        print(f"🔗 Linking ideas to conversation_id: {conversation_id}")
        
        saved_idea_ids = []
        failed_count = 0
        
        for i, scored_idea in enumerate(scored_ideas, 1):
            try:
                # Calculate total_score if not present
                if 'total_score' not in scored_idea:
                    scored_idea['total_score'] = sum([
                        scored_idea.get('usefulness_potential', 0),
                        scored_idea.get('fitwith_seo_strategy', 0),
                        scored_idea.get('fitwith_content_strategy', 0),
                        scored_idea.get('inspiration_potential', 0),
                        scored_idea.get('collaboration_potential', 0),
                        scored_idea.get('innovation', 0),
                        scored_idea.get('difficulty', 0)
                    ])
                
                blog_idea = BlogPostIdeaCreate(
                    conversation_id=conversation_id,
                    title=scored_idea.get('title', 'Untitled'),
                    description=scored_idea.get('description', ''),
                    usefulness_potential=scored_idea.get('usefulness_potential', 5),
                    fitwith_seo_strategy=scored_idea.get('fitwith_seo_strategy', 5),
                    fitwith_content_strategy=scored_idea.get('fitwith_content_strategy', 5),
                    inspiration_potential=scored_idea.get('inspiration_potential', 5),
                    collaboration_potential=scored_idea.get('collaboration_potential', 5),
                    innovation=scored_idea.get('innovation', 5),
                    difficulty=scored_idea.get('difficulty', 5),
                    sent_to_prod=False,
                    raw_llm_response=scored_idea.get('reasoning', None)
                )
                
                idea_id = db.create_blog_post_idea(blog_idea)
                saved_idea_ids.append(idea_id)
                
                print(f"   ✅ Saved idea {i}: '{scored_idea.get('title', 'Unknown')[:50]}...' (ID: {idea_id})")
                
            except Exception as e:
                print(f"   ❌ Failed to save idea {i}: {e}")
                failed_count += 1
        
        if saved_idea_ids:
            print(f"\n🎉 Database saver completed!")
            print(f"✅ Successfully saved: {len(saved_idea_ids)} ideas")
            if failed_count > 0:
                print(f"⚠️  Failed to save: {failed_count} ideas")
            
            # FIXED: Explicitly return saved_idea_ids in the state
            return {
                **state,
                "saved_idea_ids": saved_idea_ids,  # ← This is the critical line
                "status": "ideas_saved_to_db"
            }
        else:
            print("❌ Failed to save any ideas to database")
            return {
                **state,
                "saved_idea_ids": [],  # Return empty list instead of None
                "error": "Failed to save any ideas to database",
                "status": "error"
            }
            
    except Exception as e:
        print(f"❌ Error in database saver node: {e}")
        import traceback
        traceback.print_exc()
        
        return {
            **state,
            "saved_idea_ids": [],  # Return empty list on error
            "error": f"Database saver error: {str(e)}",
            "status": "error"
        }

print("✅ Database saver node (blog ideas) ready - FIXED RETURN")

✅ Database saver node (blog ideas) ready - FIXED RETURN


In [22]:
# Cell 21: Updated Pipeline Builder - Now with 5 Nodes
def build_pipeline():
    """Build the complete LangGraph workflow: Audio → Transcribe → Save → Insights → Ideas → Scoring"""
    workflow = StateGraph(AudioPipelineState)
    
    # Add all 5 nodes
    workflow.add_node("transcribe", transcription_node)
    workflow.add_node("save_to_db", database_saver_node_conversations)  
    workflow.add_node("extract_insights", pain_extractor_node)
    workflow.add_node("creative_agent", creative_agent_node)
    workflow.add_node("analyst_agent", analyst_agent_node)
    workflow.add_node("save_ideas", database_saver_node)  # NEW: Node 6
    
    # Chain them together
    workflow.add_edge("transcribe", "save_to_db")
    workflow.add_edge("save_to_db", "extract_insights")
    workflow.add_edge("extract_insights", "creative_agent")
    workflow.add_edge("creative_agent", "analyst_agent")    
    workflow.add_edge("analyst_agent", "save_ideas") 
    
    workflow.set_entry_point("transcribe")
    workflow.set_finish_point("analyst_agent")              # NEW: End with analyst
    
    return workflow.compile()

# Rebuild the pipeline with 5 nodes
pipeline = build_pipeline()
print("✅ LangGraph pipeline compiled (6 nodes: transcribe → save_to_db → extract_insights → creative_agent → analyst_agent - save scored ideas to db)")

✅ LangGraph pipeline compiled (6 nodes: transcribe → save_to_db → extract_insights → creative_agent → analyst_agent - save scored ideas to db)


In [23]:
# Cell 7: Setup Anthropic LLM for Insights Extraction (FIXED)


# Initialize Anthropic with correct model name
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
if not anthropic_key:
    print("⚠️  ANTHROPIC_API_KEY not found in .env file")
    print("Please add: ANTHROPIC_API_KEY=your_key_here")
else:
    llm = ChatAnthropic(
        model="claude-haiku-4-5",  # ← Updated model name
        api_key=anthropic_key,
        temperature=0.1
    )
    print("✅ Anthropic LLM initialized with Claude Haiku 4.5")

✅ Anthropic LLM initialized with Claude Haiku 4.5


In [24]:
## 3. PainExtractor Node Implementation

# System prompt
PAIN_EXTRACTOR_SYSTEM_PROMPT = """
You are a UX researcher and business analyst for BigKids Automation. Your job is listening to transcripts from interviews with users and potential clients. 

You pay special attention to problems that users have regarding how their company is automating, using web apps and AI to save time and move towards a more ethical and sovereign tech infrastructure.

You will be given the transcript of an interview with a user or potential client.

Your task is to extract structured information about:
- Who is speaking and their role
- What this person cares about (values, priorities)
- Their main primary and secondary challenges
- How they are solving problems today
- Are there AI agents that can assist them?
- Their underlying psychological needs (using frameworks like NVC - Non-Violent Communication)

Focus on automation, web apps, AI, time-saving, ethical tech, and sovereign infrastructure themes.

Be thorough but concise. 

IMPORTANT: Only extract information that is explicitly mentioned in the transcript. 
If information is not clearly stated, leave the field empty/null rather than guessing or inferring.
Do not hallucinate or make assumptions about missing information.
"""

In [25]:
# Cell 16: Enhanced Strategy Context for Scoring (Updated for 3 Documents)
def prepare_strategy_context_for_scoring():
    """Prepare strategy context for scoring using all three strategy documents"""
    
    # Load all three strategy documents
    strategy_context = load_company_strategy_context()
    
    # Add scoring guidelines
    strategy_context["scoring_guidelines"] = """
    SCORING CRITERIA (1-10 scale):
    
    1. usefulness_potential: How useful will this post be to readers with problems?
    2. fitwith_seo_strategy: How well does this align with our SEO strategy and keywords?
    3. fitwith_content_strategy: How well does this fit our content strategy and voice?
    4. inspiration_potential: How likely is this to inspire readers to take action?
    5. collaboration_potential: How likely is this to encourage prospects to contact us?
    6. innovation: How unique/differentiated is this topic (10 = very unique)?
    7. difficulty: How complex is this to write (1 = very complex, 10 = easy)?
    """
    
    # Create summaries for LLM prompt efficiency (all three documents)
    if strategy_context.get('company_strategy'):
        strategy_context["company_strategy_summary"] = strategy_context['company_strategy'][:800] + "..."
    
    if strategy_context.get('seo_strategy'):
        strategy_context["seo_strategy_summary"] = strategy_context['seo_strategy'][:600] + "..."
    
    if strategy_context.get('content_strategy'):  # NEW
        strategy_context["content_strategy_summary"] = strategy_context['content_strategy'][:600] + "..."
    
    print(f"✅ Enhanced strategy context for scoring with 3 documents")
    print(f"   Company strategy: {len(strategy_context.get('company_strategy', ''))} chars")
    print(f"   SEO strategy: {len(strategy_context.get('seo_strategy', ''))} chars")
    print(f"   Content strategy: {len(strategy_context.get('content_strategy', ''))} chars")
    
    return strategy_context

# Test the enhanced context
enhanced_context = prepare_strategy_context_for_scoring()

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
✅ Enhanced strategy context for scoring with 3 documents
   Company strategy: 6555 chars
   SEO strategy: 1120 chars
   Content strategy: 4469 chars


In [26]:
# Cell: Fixed Test Function (No Duplicate Loading)
def test_three_document_loading():
    """Test loading all three strategy documents (optimized)"""
    
    print("🧪 Testing three-document strategy loading...")
    
    # Load documents once and enhance
    enhanced = prepare_strategy_context_for_scoring()  # This calls load_company_strategy_context() internally
    
    print(f"\n📊 DOCUMENT SUMMARY:")
    for doc_type in ['company_strategy', 'seo_strategy', 'content_strategy']:
        if doc_type in enhanced:
            length = len(enhanced[doc_type]) if enhanced[doc_type] else 0
            status = "✅ Loaded" if length > 100 else "⚠️ Missing/Short"
            print(f"   {doc_type}: {status} ({length} chars)")
    
    return enhanced

# Test with no duplicates
test_context = test_three_document_loading()

🧪 Testing three-document strategy loading...
✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
✅ Enhanced strategy context for scoring with 3 documents
   Company strategy: 6555 chars
   SEO strategy: 1120 chars
   Content strategy: 4469 chars

📊 DOCUMENT SUMMARY:
   company_strategy: ✅ Loaded (6555 chars)
   seo_strategy: ✅ Loaded (1120 chars)
   content_strategy: ✅ Loaded (4469 chars)


In [37]:
# ============================================================
# CELL 22: EXECUTE COMPLETE 6-NODE PIPELINE TEST (FIXED)
# ============================================================

def test_complete_6_node_pipeline():
    """Test the complete 6-node pipeline: Audio → Transcribe → Save → Insights → Ideas → Scoring → Save Ideas"""
    
    print("🧪 EXECUTING COMPLETE 6-NODE PIPELINE TEST...")
    print("Audio → Transcribe → Save → Insights → Ideas → Scoring → Save Ideas")
    print("-" * 60)
    
    # Find audio files
    audio_files = find_audio_files_in_temp()
    
    if not audio_files:
        print("❌ No audio files found in data/temp/")
        print("💡 Add .wav files to data/temp/ and run this cell again")
        return None
    
    print(f"📁 Found {len(audio_files)} audio file(s)")
    print(f"🎯 Testing with: {audio_files[0].name}")
    print(f"📊 File size: {audio_files[0].stat().st_size / 1024:.1f} KB")
    
    # Create initial state for 6-node pipeline
    initial_state = {
        "file_path": str(audio_files[0]),
        "filename": audio_files[0].name,
        "transcript_text": None,
        "conversation_id": None,
        "extracted_insights": None,
        "raw_blog_ideas": None,
        "scored_blog_ideas": None,
        "saved_idea_ids": None,  # NEW: For Node 6
        "error": None,
        "status": "processing"
    }
    
    try:
        print("\n🎬 STARTING COMPLETE PIPELINE EXECUTION...")
        print("=" * 60)
        
        # Run the complete 6-node pipeline
        final_state = pipeline.invoke(initial_state)
        
        print(f"\n📊 COMPLETE PIPELINE RESULTS:")
        print("=" * 60)
        print(f"🎯 Final Status: {final_state.get('status')}")
        print(f"📝 Conversation ID: {final_state.get('conversation_id')}")
        print(f"💾 Saved Blog Idea IDs: {final_state.get('saved_idea_ids')}")
        
        # Check all pipeline stages
        print(f"\n📋 STAGE RESULTS:")
        stages = [
            ("🎙️  Transcription", final_state.get('transcript_text')),
            ("💾 Database Save (Conversation)", final_state.get('conversation_id')),
            ("🧠 Insights Extraction", final_state.get('extracted_insights')),
            ("🎨 Blog Ideas Generation", final_state.get('raw_blog_ideas')),
            ("🔍 Blog Ideas Scoring", final_state.get('scored_blog_ideas')),
            ("💾 Database Save (Ideas)", final_state.get('saved_idea_ids'))  # NEW: Node 6
        ]
        
        all_passed = True
        for stage_name, stage_data in stages:
            status = "✅" if stage_data else "❌"
            print(f"   {stage_name}: {status}")
            if not stage_data:
                all_passed = False
        
        # Show detailed results if all stages passed
        if all_passed and final_state.get('scored_blog_ideas') and final_state.get('saved_idea_ids'):
            scored_ideas = final_state['scored_blog_ideas']
            saved_ids = final_state['saved_idea_ids']
            
            print(f"\n🎉 COMPLETE SUCCESS! Pipeline generated, scored, and saved {len(saved_ids)} blog ideas")
            print("=" * 80)
            
            # Show conversation info
            print(f"\n📝 Conversation Details:")
            print(f"   ID: {final_state.get('conversation_id')}")
            print(f"   Transcript Length: {len(final_state.get('transcript_text', ''))} characters")
            
            # Show insights summary
            if final_state.get('extracted_insights'):
                insights = final_state['extracted_insights']
                print(f"\n🧠 Extracted Insights:")
                print(f"   👥 Speakers: {len(insights.speakers) if hasattr(insights, 'speakers') else 'N/A'}")
                print(f"   🔥 Primary Challenges: {len(insights.primary_challenges) if hasattr(insights, 'primary_challenges') else 'N/A'}")
                print(f"   🧘 Psychological Needs: {len(insights.psychological_needs) if hasattr(insights, 'psychological_needs') else 'N/A'}")
                print(f"   💎 Core Values: {len(insights.core_values) if hasattr(insights, 'core_values') else 'N/A'}")
            
            # Show database save results
            print(f"\n💾 Database Save Summary:")
            print(f"   Total Ideas Generated: {len(scored_ideas)}")
            print(f"   Ideas Successfully Saved: {len(saved_ids)}")
            print(f"   Save Success Rate: {len(saved_ids)/len(scored_ideas)*100:.1f}%")
            print(f"   Saved Idea IDs: {saved_ids}")
            
            # Show top scored ideas with their database IDs
            print(f"\n🎨 Top Scored Blog Ideas (Now in Database!):")
            print("-" * 80)
            
            # FIXED: Proper zip syntax
            top_count = min(3, len(scored_ideas), len(saved_ids))  # Get minimum to avoid index errors
            for i in range(top_count):
                idea = scored_ideas[i]
                db_id = saved_ids[i]
                
                print(f"\n🏆 RANKED IDEA #{i+1} (Total Score: {idea.get('total_score', 0)}/70):")
                print(f"   🆔 Database ID: {db_id}")
                print(f"   📝 Title: {idea.get('title', 'No title')[:75]}...")
                print(f"   📖 Description: {idea.get('description', 'No description')[:100]}...")
                print(f"   📊 Breakdown:")
                print(f"      • Usefulness: {idea.get('usefulness_potential', 0)}/10")
                print(f"      • SEO Fit: {idea.get('fitwith_seo_strategy', 0)}/10")
                print(f"      • Content Fit: {idea.get('fitwith_content_strategy', 0)}/10")
                print(f"      • Inspiration: {idea.get('inspiration_potential', 0)}/10")
                print(f"      • Collaboration: {idea.get('collaboration_potential', 0)}/10")
                print(f"      • Innovation: {idea.get('innovation', 0)}/10")
                print(f"      • Easy to Write: {idea.get('difficulty', 0)}/10")
            
            print("=" * 80)
            print("🎉 COMPLETE 6-NODE PIPELINE: SUCCESS!")
            print("✅ All stages completed successfully")
            print(f"💾 Conversation saved (ID: {final_state.get('conversation_id')})")
            print(f"💾 {len(saved_ids)} blog ideas saved to database")
            print(f"🔗 Ideas linked to conversation for traceability")
            
            # Show how to retrieve the saved data
            print(f"\n💡 NEXT STEPS:")
            print(f"   • Query saved ideas: db.get_blog_post_ideas_by_conversation({final_state.get('conversation_id')})")
            print(f"   • View conversation: db.get_conversation({final_state.get('conversation_id')})")
            print(f"   • Access specific idea: db.get_blog_post_idea({saved_ids[0]})")
            
        else:
            # Something failed
            print("\n❌ PIPELINE INCOMPLETE")
            print("=" * 50)
            if final_state.get('error'):
                print(f"❌ Error: {final_state.get('error')}")
            else:
                print("❌ Pipeline stopped but no error message provided")
            
            # Show what we did get
            print(f"\n🔍 DEBUG INFO:")
            print(f"   Transcript exists: {bool(final_state.get('transcript_text'))}")
            if final_state.get('transcript_text'):
                print(f"   Transcript preview: {final_state.get('transcript_text')[:100]}...")
            print(f"   Conversation ID: {final_state.get('conversation_id')}")
            print(f"   Insights exist: {bool(final_state.get('extracted_insights'))}")
            print(f"   Raw ideas exist: {bool(final_state.get('raw_blog_ideas'))}")
            if final_state.get('raw_blog_ideas'):
                print(f"   Raw ideas count: {len(final_state.get('raw_blog_ideas'))}")
            print(f"   Scored ideas exist: {bool(final_state.get('scored_blog_ideas'))}")
            if final_state.get('scored_blog_ideas'):
                print(f"   Scored ideas count: {len(final_state.get('scored_blog_ideas'))}")
            print(f"   Saved idea IDs exist: {bool(final_state.get('saved_idea_ids'))}")
            if final_state.get('saved_idea_ids'):
                print(f"   Saved ideas count: {len(final_state.get('saved_idea_ids'))}")
        
        return final_state
        
    except Exception as e:
        print(f"\n❌ COMPLETE PIPELINE FAILED WITH EXCEPTION:")
        print(f"   {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# ============================================================
# RUN THE TEST
# ============================================================

print("🚀 Ready to test complete 6-node pipeline")
print("💡 Run the cell below to execute the test\n")

# Uncomment the line below to run automatically, or run it manually
test_result = test_complete_6_node_pipeline()


🚀 Ready to test complete 6-node pipeline
💡 Run the cell below to execute the test

🧪 EXECUTING COMPLETE 6-NODE PIPELINE TEST...
Audio → Transcribe → Save → Insights → Ideas → Scoring → Save Ideas
------------------------------------------------------------
📁 Found 1 audio file(s)
🎯 Testing with: blog_record_(purevitalize).wav
📊 File size: 25016.7 KB

🎬 STARTING COMPLETE PIPELINE EXECUTION...
🎙️ Transcribing: blog_record_(purevitalize).wav
💾 Saving to database: blog_record_(purevitalize).wav
🧠 Starting pain extraction...
📝 Raw response length: 3201 chars
📝 Response starts with: ```json
{
    "speakers": [
        {
            ...
🔧 Removing JSON markdown blocks...
🔧 Cleaned content starts with: {
    "speakers": [
        {
            "name": ...
✅ Successfully extracted insights with correct speaker roles!
✅ Extracted insights: 3 primary challenges, 1 speakers
🎨 Starting creative blog idea generation...
📊 Working with insights: 3 challenges
✅ Loaded company strategy (6555 chars)
✅ Lo

/tmp/ipykernel_6381/4162762669.py:36: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  ideas_as_dicts = [idea.dict() for idea in validated_ideas]


   ✅ Scored: 47/70 points
🔍 Scoring idea 2/5: The Security Audit Checklist: What to Ask Before I...
   ✅ Scored: 43/70 points
🔍 Scoring idea 3/5: Email Automation Without the Risk: How to Implemen...
   ✅ Scored: 43/70 points
🔍 Scoring idea 4/5: From Uncertainty to Strategy: How to Build Your AI...
   ✅ Scored: 43/70 points
🔍 Scoring idea 5/5: The Interdependence Model: How to Choose AI Soluti...
   ✅ Scored: 56/70 points

🎉 Analyst agent completed scoring!
📊 Scored 5 ideas
🏆 Top idea: 'The Interdependence Model: How to Choose AI Soluti...' (56/70)
📉 Lowest idea: 'From Uncertainty to Strategy: How to Build Your AI...' (43/70)
💾 Starting database saver - saving scored blog ideas...
📊 Found 5 scored ideas to save
🔗 Linking ideas to conversation_id: 28
   ✅ Saved idea 1: 'The Interdependence Model: How to Choose AI Soluti...' (ID: 13)
   ✅ Saved idea 2: 'Custom AI vs SaaS for GDPR Compliance: A Decision ...' (ID: 14)
   ✅ Saved idea 3: 'The Security Audit Checklist: What to Ask Before I..

In [27]:
# Cell: Rebuild Pipeline with Updated Node
pipeline = build_pipeline()
print("✅ Pipeline rebuilt with updated creative agent")

✅ Pipeline rebuilt with updated creative agent


In [31]:
# Cell: Terminal Dashboard - Using Existing DB Methods
def show_ideas_dashboard(conversation_id=None, top_n=10):
    """
    Beautiful terminal dashboard showing scored blog ideas
    Uses existing db.get_all_ideas() or db.get_ideas_by_conversation()
    
    Args:
        conversation_id: Show ideas for specific conversation (None = all ideas)
        top_n: Number of top ideas to show
    """
    
    print("\n" + "=" * 100)
    print("📊 BLOG IDEAS DASHBOARD")
    print("=" * 100)
    
    # Get ideas using existing methods
    if conversation_id:
        ideas = db.get_ideas_by_conversation(conversation_id)
        print(f"📁 Showing ideas from Conversation ID: {conversation_id}")
    else:
        ideas = db.get_all_ideas()
        print(f"📁 Showing ALL ideas from database")
    
    if not ideas:
        print("⚠️  No ideas found in database")
        return
    
    # Calculate scores and sort
    scored_ideas = []
    for idea in ideas:
        # Handle both dict and object formats
        if isinstance(idea, dict):
            total = sum([
                idea.get('usefulness_potential', 0),
                idea.get('fitwith_seo_strategy', 0),
                idea.get('fitwith_content_strategy', 0),
                idea.get('inspiration_potential', 0),
                idea.get('collaboration_potential', 0),
                idea.get('innovation', 0),
                idea.get('difficulty', 0)
            ])
        else:
            total = sum([
                idea.usefulness_potential,
                idea.fitwith_seo_strategy,
                idea.fitwith_content_strategy,
                idea.inspiration_potential,
                idea.collaboration_potential,
                idea.innovation,
                idea.difficulty
            ])
        scored_ideas.append((idea, total))
    
    # Sort by total score (highest first)
    scored_ideas.sort(key=lambda x: x[1], reverse=True)
    
    # Show summary stats
    all_scores = [s[1] for s in scored_ideas]
    avg_score = sum(all_scores) / len(all_scores)
    
    print(f"\n📈 SUMMARY STATISTICS")
    print(f"   Total Ideas: {len(scored_ideas)}")
    print(f"   Average Score: {avg_score:.1f}/70 ({avg_score/70*100:.1f}%)")
    print(f"   Highest Score: {scored_ideas[0][1]}/70 ({scored_ideas[0][1]/70*100:.1f}%)")
    print(f"   Lowest Score: {scored_ideas[-1][1]}/70 ({scored_ideas[-1][1]/70*100:.1f}%)")
    
    # Show score distribution
    high_scores = sum(1 for s in all_scores if s >= 60)
    medium_scores = sum(1 for s in all_scores if 50 <= s < 60)
    low_scores = sum(1 for s in all_scores if s < 50)
    
    print(f"\n📊 SCORE DISTRIBUTION")
    print(f"   🟢 High (60-70):  {high_scores} ideas ({high_scores/len(all_scores)*100:.1f}%)")
    print(f"   🟡 Medium (50-59): {medium_scores} ideas ({medium_scores/len(all_scores)*100:.1f}%)")
    print(f"   🔴 Low (<50):     {low_scores} ideas ({low_scores/len(all_scores)*100:.1f}%)")
    
    # Show top ideas
    display_count = min(top_n, len(scored_ideas))
    print(f"\n🏆 TOP {display_count} IDEAS")
    print("-" * 100)
    
    for rank, (idea, total_score) in enumerate(scored_ideas[:top_n], 1):
        # Handle both dict and object formats
        if isinstance(idea, dict):
            idea_id = idea.get('id')
            title = idea.get('title', 'Untitled')
            usefulness = idea.get('usefulness_potential', 0)
            seo = idea.get('fitwith_seo_strategy', 0)
            content = idea.get('fitwith_content_strategy', 0)
            inspiration = idea.get('inspiration_potential', 0)
            collaboration = idea.get('collaboration_potential', 0)
            innovation = idea.get('innovation', 0)
            difficulty = idea.get('difficulty', 0)
            created_at = idea.get('created_at', 'Unknown')
            conv_id = idea.get('conversation_id', 'N/A')
            sent_to_prod = idea.get('sent_to_prod', False)
        else:
            idea_id = idea.id
            title = idea.title
            usefulness = idea.usefulness_potential
            seo = idea.fitwith_seo_strategy
            content = idea.fitwith_content_strategy
            inspiration = idea.inspiration_potential
            collaboration = idea.collaboration_potential
            innovation = idea.innovation
            difficulty = idea.difficulty
            created_at = idea.created_at
            conv_id = idea.conversation_id
            sent_to_prod = idea.sent_to_prod
        
        # Create score bar
        bar_length = 35
        percentage = total_score / 70
        filled = int(percentage * bar_length)
        bar = "█" * filled + "░" * (bar_length - filled)
        
        # Medal emoji for top 3
        medal = {1: "🥇", 2: "🥈", 3: "🥉"}.get(rank, f"{rank:2d}.")
        
        # Color indicator based on score
        if total_score >= 60:
            indicator = "🟢"  # High score
        elif total_score >= 50:
            indicator = "🟡"  # Medium score
        else:
            indicator = "🔴"  # Low score
        
        print(f"\n{medal} {indicator} ID: {idea_id} | Score: {total_score}/70 ({percentage*100:.1f}%)")
        print(f"   📝 {title}")
        print(f"   📊 [{bar}] {total_score}/70")
        print(f"   💡 Breakdown:")
        print(f"      • Usefulness: {usefulness}/10 | SEO Fit: {seo}/10 | Content Fit: {content}/10")
        print(f"      • Inspiration: {inspiration}/10 | Collaboration: {collaboration}/10")
        print(f"      • Innovation: {innovation}/10 | Difficulty (ease): {difficulty}/10")
        print(f"   📅 Created: {created_at}")
        print(f"   🔗 Conversation: {conv_id}")
        
        if sent_to_prod:
            print(f"   ✅ STATUS: SENT TO PRODUCTION")
        else:
            print(f"   📝 STATUS: Draft")
    
    print("\n" + "=" * 100)
    print("💡 USAGE TIPS:")
    print("   show_ideas_dashboard()              # Show all ideas")
    print("   show_ideas_dashboard(28)            # Show ideas from conversation 28")
    print("   show_ideas_dashboard(28, top_n=3)   # Show top 3 ideas only")
    print("=" * 100 + "\n")

print("✅ Terminal dashboard ready (using db.get_all_ideas and db.get_ideas_by_conversation)")

✅ Terminal dashboard ready (using db.get_all_ideas and db.get_ideas_by_conversation)


In [32]:
# Cell: Quick View - Compact Dashboard
def quick_view(conversation_id=None):
    """Quick compact view of scored ideas"""
    
    ideas = db.get_ideas_by_conversation(conversation_id) if conversation_id else db.get_all_ideas()
    
    if not ideas:
        print("⚠️  No ideas found")
        return
    
    # Score and sort
    scored = []
    for idea in ideas:
        if isinstance(idea, dict):
            total = sum([idea.get('usefulness_potential', 0), idea.get('fitwith_seo_strategy', 0),
                        idea.get('fitwith_content_strategy', 0), idea.get('inspiration_potential', 0),
                        idea.get('collaboration_potential', 0), idea.get('innovation', 0), idea.get('difficulty', 0)])
            scored.append((idea, total))
        else:
            total = sum([idea.usefulness_potential, idea.fitwith_seo_strategy, idea.fitwith_content_strategy,
                        idea.inspiration_potential, idea.collaboration_potential, idea.innovation, idea.difficulty])
            scored.append((idea, total))
    
    scored.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n{'='*100}")
    print(f"📊 {'CONVERSATION ' + str(conversation_id) if conversation_id else 'ALL IDEAS'} | Total: {len(scored)} ideas | Avg: {sum(s[1] for s in scored)/len(scored):.1f}/70")
    print(f"{'='*100}\n")
    
    for rank, (idea, score) in enumerate(scored, 1):
        if isinstance(idea, dict):
            idea_id, title = idea.get('id'), idea.get('title', 'Untitled')
        else:
            idea_id, title = idea.id, idea.title
        
        medal = {1: "🥇", 2: "🥈", 3: "🥉"}.get(rank, f"{rank:2d}.")
        indicator = "🟢" if score >= 60 else "🟡" if score >= 50 else "🔴"
        
        print(f"{medal} {indicator} [{idea_id:3d}] {score:2d}/70 | {title[:75]}")
    
    print(f"\n{'='*100}\n")

print("✅ Quick view function ready")

✅ Quick view function ready


In [33]:
# Test 1: Full dashboard for conversation 28
print("🧪 Test 1: Full dashboard for conversation 28")
show_ideas_dashboard(28)

# Test 2: Quick view for all ideas
print("\n🧪 Test 2: Quick view for all ideas")
quick_view()

# Test 3: Quick view for conversation 28
print("\n🧪 Test 3: Quick view for conversation 28")
quick_view(28)

🧪 Test 1: Full dashboard for conversation 28

📊 BLOG IDEAS DASHBOARD
📁 Showing ideas from Conversation ID: 28

📈 SUMMARY STATISTICS
   Total Ideas: 5
   Average Score: 46.4/70 (66.3%)
   Highest Score: 56/70 (80.0%)
   Lowest Score: 43/70 (61.4%)

📊 SCORE DISTRIBUTION
   🟢 High (60-70):  0 ideas (0.0%)
   🟡 Medium (50-59): 1 ideas (20.0%)
   🔴 Low (<50):     4 ideas (80.0%)

🏆 TOP 5 IDEAS
----------------------------------------------------------------------------------------------------

🥇 🟡 ID: 13 | Score: 56/70 (80.0%)
   📝 The Interdependence Model: How to Choose AI Solutions That Align With Your Company Values
   📊 [████████████████████████████░░░░░░░] 56/70
   💡 Breakdown:
      • Usefulness: 9/10 | SEO Fit: 7/10 | Content Fit: 10/10
      • Inspiration: 8/10 | Collaboration: 9/10
      • Innovation: 8/10 | Difficulty (ease): 5/10
   📅 Created: 2025-11-28 09:03:16
   🔗 Conversation: 28
   📝 STATUS: Draft

🥈 🔴 ID: 14 | Score: 47/70 (67.1%)
   📝 Custom AI vs SaaS for GDPR Complianc

In [46]:
# Cell: FIXED list_conversations (Run this to replace the old one)
def list_conversations():
    """Show all conversations in the database (safe version - NO audio_file_path)"""
    
    conversations = db.get_all_conversations()
    
    if not conversations:
        print("⚠️  No conversations found in database")
        return
    
    print("\n" + "=" * 100)
    print("💬 ALL CONVERSATIONS IN DATABASE")
    print("=" * 100)
    
    for conv in conversations:
        # Use the safe helper functions (NO direct attribute access)
        conv_id = get_conv_attribute(conv, 'id', 'Unknown')
        filename = get_conv_filename(conv)  # This handles missing audio_file_path
        transcript = get_conv_attribute(conv, 'transcript_text', '')
        transcript_len = len(transcript) if transcript else 0
        created = get_conv_attribute(conv, 'created_at', 'Unknown')
        
        # Get idea count for this conversation
        ideas = db.get_ideas_by_conversation(conv_id)
        idea_count = len(ideas) if ideas else 0
        
        # Calculate average score if ideas exist
        if ideas and idea_count > 0:
            total_scores = []
            for idea in ideas:
                if isinstance(idea, dict):
                    score = sum([idea.get('usefulness_potential', 0), idea.get('fitwith_seo_strategy', 0),
                               idea.get('fitwith_content_strategy', 0), idea.get('inspiration_potential', 0),
                               idea.get('collaboration_potential', 0), idea.get('innovation', 0), 
                               idea.get('difficulty', 0)])
                else:
                    score = sum([
                        get_idea_attribute(idea, 'usefulness_potential', 0),
                        get_idea_attribute(idea, 'fitwith_seo_strategy', 0),
                        get_idea_attribute(idea, 'fitwith_content_strategy', 0),
                        get_idea_attribute(idea, 'inspiration_potential', 0),
                        get_idea_attribute(idea, 'collaboration_potential', 0),
                        get_idea_attribute(idea, 'innovation', 0),
                        get_idea_attribute(idea, 'difficulty', 0)
                    ])
                total_scores.append(score)
            avg_score = sum(total_scores) / len(total_scores)
            score_info = f"Avg Score: {avg_score:.1f}/70"
        else:
            score_info = "No ideas yet"
        
        print(f"\n📁 ID: {conv_id}")
        print(f"   🎙️  File: {filename}")
        print(f"   📝 Transcript: {transcript_len} characters")
        print(f"   💡 Ideas: {idea_count} | {score_info}")
        print(f"   📅 Created: {created}")
        print(f"   🔗 View: show_ideas_dashboard({conv_id})")
    
    print("\n" + "=" * 100)
    print(f"💡 Total Conversations: {len(conversations)}")
    print("=" * 100 + "\n")

print("✅ list_conversations() REPLACED with safe version")

✅ list_conversations() REPLACED with safe version


In [36]:
# Cell: Interactive Conversation Selector
def select_conversation():
    """
    Interactive menu to select a conversation to visualize
    Returns the selected conversation ID
    """
    
    conversations = db.get_all_conversations()
    
    if not conversations:
        print("⚠️  No conversations found in database")
        return None
    
    print("\n" + "=" * 80)
    print("🔍 SELECT A CONVERSATION TO VISUALIZE")
    print("=" * 80 + "\n")
    
    # Create a list with conversation details
    conv_list = []
    for conv in conversations:
        if isinstance(conv, dict):
            conv_id = conv.get('id')
            filename = conv.get('audio_file_path', 'Unknown')
            created = conv.get('created_at', 'Unknown')
        else:
            conv_id = conv.id
            filename = conv.audio_file_path
            created = conv.created_at
        
        # Get idea count
        ideas = db.get_ideas_by_conversation(conv_id)
        idea_count = len(ideas) if ideas else 0
        
        conv_list.append((conv_id, filename, idea_count, created))
    
    # Sort by ID (most recent first)
    conv_list.sort(key=lambda x: x[0], reverse=True)
    
    # Display options
    for i, (conv_id, filename, idea_count, created) in enumerate(conv_list, 1):
        print(f"{i}. [ID: {conv_id}] {filename}")
        print(f"   💡 {idea_count} ideas | 📅 {created}")
        print()
    
    print("=" * 80)
    
    # Get user choice
    try:
        choice = input(f"Enter number (1-{len(conv_list)}) or 'q' to quit: ").strip()
        
        if choice.lower() == 'q':
            print("❌ Cancelled")
            return None
        
        choice_num = int(choice)
        if 1 <= choice_num <= len(conv_list):
            selected_id = conv_list[choice_num - 1][0]
            print(f"\n✅ Selected Conversation ID: {selected_id}")
            return selected_id
        else:
            print(f"❌ Invalid choice. Please enter 1-{len(conv_list)}")
            return None
            
    except ValueError:
        print("❌ Invalid input. Please enter a number.")
        return None
    except KeyboardInterrupt:
        print("\n❌ Cancelled")
        return None

print("✅ select_conversation() ready")

✅ select_conversation() ready


In [37]:
# Cell: Smart Dashboard with Auto-Detection
def smart_dashboard(conversation_id=None, mode='latest'):
    """
    Smart dashboard that can auto-select conversation
    
    Args:
        conversation_id: Specific conversation ID (overrides mode)
        mode: 'latest' = most recent, 'all' = all ideas, 'best' = highest scoring conversation
    """
    
    if conversation_id:
        # User specified ID
        show_ideas_dashboard(conversation_id)
        return
    
    conversations = db.get_all_conversations()
    
    if not conversations:
        print("⚠️  No conversations found")
        return
    
    if mode == 'latest':
        # Show most recent conversation
        latest = max(conversations, key=lambda c: c.id if not isinstance(c, dict) else c.get('id'))
        latest_id = latest.id if not isinstance(latest, dict) else latest.get('id')
        print(f"📌 Showing LATEST conversation (ID: {latest_id})")
        show_ideas_dashboard(latest_id)
        
    elif mode == 'best':
        # Find conversation with highest average score
        best_conv_id = None
        best_avg = 0
        
        for conv in conversations:
            conv_id = conv.id if not isinstance(conv, dict) else conv.get('id')
            ideas = db.get_ideas_by_conversation(conv_id)
            
            if ideas:
                scores = []
                for idea in ideas:
                    if isinstance(idea, dict):
                        score = sum([idea.get('usefulness_potential', 0), idea.get('fitwith_seo_strategy', 0),
                                   idea.get('fitwith_content_strategy', 0), idea.get('inspiration_potential', 0),
                                   idea.get('collaboration_potential', 0), idea.get('innovation', 0), 
                                   idea.get('difficulty', 0)])
                    else:
                        score = sum([idea.usefulness_potential, idea.fitwith_seo_strategy, idea.fitwith_content_strategy,
                                   idea.inspiration_potential, idea.collaboration_potential, idea.innovation, idea.difficulty])
                    scores.append(score)
                
                avg = sum(scores) / len(scores)
                if avg > best_avg:
                    best_avg = avg
                    best_conv_id = conv_id
        
        if best_conv_id:
            print(f"📌 Showing BEST conversation (ID: {best_conv_id}, Avg: {best_avg:.1f}/70)")
            show_ideas_dashboard(best_conv_id)
        else:
            print("⚠️  No conversations with ideas found")
            
    elif mode == 'all':
        # Show all ideas
        print(f"📌 Showing ALL ideas from all conversations")
        show_ideas_dashboard()
    
    else:
        print(f"❌ Unknown mode: {mode}. Use 'latest', 'best', or 'all'")

print("✅ smart_dashboard() ready")

✅ smart_dashboard() ready


In [52]:
# ============================================
# SIMPLE: List all conversations and choose
# ============================================

# Step 1: See what conversations you have
list_conversations()

# Step 2: Pick one and visualize
show_ideas_dashboard(28)  # Replace 28 with your chosen ID


# ============================================
# INTERACTIVE: Let the system guide you
# ============================================

# Interactive selection
selected_id = select_conversation()
if selected_id:
    show_ideas_dashboard(selected_id)


# ============================================
# SMART: Auto-select interesting conversations
# ============================================

# Show latest conversation
smart_dashboard(mode='latest')

# Show conversation with best scores
smart_dashboard(mode='best')

# Show all ideas
smart_dashboard(mode='all')

# Or specify exact ID
smart_dashboard(conversation_id=28)


💬 ALL CONVERSATIONS IN DATABASE

📁 ID: 28
   🎙️  File: Conversation_28_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 46.4/70
   📅 Created: 2025-11-28 09:02:37
   🔗 View: show_ideas_dashboard(28)

📁 ID: 27
   🎙️  File: Conversation_27_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 35.0/70
   📅 Created: 2025-11-28 08:27:30
   🔗 View: show_ideas_dashboard(27)

📁 ID: 26
   🎙️  File: Conversation_26_2025-11-27
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 35.0/70
   📅 Created: 2025-11-27 09:11:49
   🔗 View: show_ideas_dashboard(26)

📁 ID: 25
   🎙️  File: Conversation_25_2025-11-26
   📝 Transcript: 0 characters
   💡 Ideas: 0 | No ideas yet
   📅 Created: 2025-11-26 09:11:52
   🔗 View: show_ideas_dashboard(25)

📁 ID: 24
   🎙️  File: Conversation_24_2025-11-21
   📝 Transcript: 0 characters
   💡 Ideas: 0 | No ideas yet
   📅 Created: 2025-11-21 09:58:56
   🔗 View: show_ideas_dashboard(24)

📁 ID: 23
   🎙️  File: Conversation_23_2025-11-19
   📝

AttributeError: 'Conversation' object has no attribute 'audio_file_path'

In [47]:
# Cell: FIXED explore_and_visualize (Run this to replace)
def explore_and_visualize():
    """Complete workflow: list conversations, select, and visualize (SAFE version)"""
    
    print("🔍 STEP 1: Available Conversations")
    print("=" * 80)
    
    conversations = db.get_all_conversations()
    
    if not conversations:
        print("⚠️  No conversations found")
        return
    
    # Show conversation list
    conv_list = []
    for conv in conversations:
        # Use safe helpers (NO direct attribute access)
        conv_id = get_conv_attribute(conv, 'id', 'Unknown')
        filename = get_conv_filename(conv)  # This handles missing audio_file_path
        
        ideas = db.get_ideas_by_conversation(conv_id)
        idea_count = len(ideas) if ideas else 0
        
        conv_list.append((conv_id, filename, idea_count))
        print(f"{len(conv_list)}. [ID: {conv_id}] {filename} ({idea_count} ideas)")
    
    print("\n" + "=" * 80)
    print("🎯 STEP 2: Choose a conversation")
    print("=" * 80)
    
    try:
        choice = input(f"\nEnter number (1-{len(conv_list)}), 'all' for everything, or 'q' to quit: ").strip()
        
        if choice.lower() == 'q':
            print("❌ Cancelled")
            return
        
        if choice.lower() == 'all':
            print("\n📊 Showing all ideas...")
            show_ideas_dashboard()
        else:
            choice_num = int(choice)
            if 1 <= choice_num <= len(conv_list):
                selected_id = conv_list[choice_num - 1][0]
                print(f"\n📊 Showing conversation {selected_id}...")
                show_ideas_dashboard(selected_id)
            else:
                print(f"❌ Invalid choice")
    
    except ValueError:
        print("❌ Invalid input")
    except KeyboardInterrupt:
        print("\n❌ Cancelled")

print("✅ explore_and_visualize() REPLACED with safe version")

✅ explore_and_visualize() REPLACED with safe version


In [51]:
# Easiest way - interactive explorer
explore_and_visualize()

# Or just list them first
list_conversations()

🔍 STEP 1: Available Conversations
1. [ID: 28] Conversation_28_2025-11-28 (5 ideas)
2. [ID: 27] Conversation_27_2025-11-28 (5 ideas)
3. [ID: 26] Conversation_26_2025-11-27 (5 ideas)
4. [ID: 25] Conversation_25_2025-11-26 (0 ideas)
5. [ID: 24] Conversation_24_2025-11-21 (0 ideas)
6. [ID: 23] Conversation_23_2025-11-19 (0 ideas)
7. [ID: 22] Conversation_22_2025-11-19 (0 ideas)
8. [ID: 21] Conversation_21_2025-11-19 (0 ideas)
9. [ID: 20] Conversation_20_2025-11-17 (0 ideas)
10. [ID: 19] Conversation_19_2025-11-17 (0 ideas)

🎯 STEP 2: Choose a conversation

📊 Showing conversation 24...

📊 BLOG IDEAS DASHBOARD
📁 Showing ideas from Conversation ID: 24
⚠️  No ideas found in database

💬 ALL CONVERSATIONS IN DATABASE

📁 ID: 28
   🎙️  File: Conversation_28_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 46.4/70
   📅 Created: 2025-11-28 09:02:37
   🔗 View: show_ideas_dashboard(28)

📁 ID: 27
   🎙️  File: Conversation_27_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Av

In [41]:
# Cell: Inspect Conversation Model
from database.models import Conversation

# Check what fields Conversation actually has
print("🔍 Inspecting Conversation model...")
print("=" * 60)

# Try to get a sample conversation
conversations = db.get_all_conversations()
if conversations:
    sample = conversations[0]
    print(f"✅ Sample Conversation:")
    print(f"   Type: {type(sample)}")
    
    if isinstance(sample, dict):
        print(f"   Keys: {list(sample.keys())}")
    else:
        # It's a Pydantic model
        print(f"   Fields: {list(sample.model_fields.keys())}")
        print(f"\n📋 Field Details:")
        for field_name, field_info in sample.model_fields.items():
            print(f"      • {field_name}: {field_info.annotation}")
else:
    print("⚠️  No conversations found to inspect")
    print("\n💡 Checking Conversation model definition instead...")
    print(f"   Fields: {list(Conversation.model_fields.keys())}")

🔍 Inspecting Conversation model...
✅ Sample Conversation:
   Type: <class 'database.models.Conversation'>
   Fields: ['id', 'title', 'raw_text', 'source', 'word_count', 'created_at', 'status']

📋 Field Details:
      • id: <class 'int'>
      • title: typing.Optional[str]
      • raw_text: <class 'str'>
      • source: <class 'str'>
      • word_count: <class 'int'>
      • created_at: <class 'datetime.datetime'>
      • status: <class 'str'>


/tmp/ipykernel_7593/1689298502.py:19: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  print(f"   Fields: {list(sample.model_fields.keys())}")
/tmp/ipykernel_7593/1689298502.py:21: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in sample.model_fields.items():


In [42]:
# Cell: Inspect Conversation Model (Pydantic v2.11 compatible)
from database.models import Conversation

print("🔍 Inspecting Conversation model...")
print("=" * 60)

# Access model_fields from the CLASS, not the instance
print(f"✅ Conversation Model Fields:")
print(f"   Fields: {list(Conversation.model_fields.keys())}")
print(f"\n📋 Field Details:")
for field_name, field_info in Conversation.model_fields.items():
    print(f"      • {field_name}: {field_info.annotation}")

# Try to get a sample conversation to see actual data
conversations = db.get_all_conversations()
if conversations:
    sample = conversations[0]
    print(f"\n✅ Sample Conversation Data:")
    print(f"   Type: {type(sample)}")
    
    if isinstance(sample, dict):
        print(f"   Keys: {list(sample.keys())}")
        print(f"\n   Sample Values:")
        for key, value in sample.items():
            if isinstance(value, str) and len(value) > 50:
                print(f"      • {key}: {value[:50]}...")
            else:
                print(f"      • {key}: {value}")
    else:
        # It's a Pydantic model - use model_dump() instead of accessing fields
        data = sample.model_dump()
        print(f"   Available attributes: {list(data.keys())}")
        print(f"\n   Sample Values:")
        for key, value in data.items():
            if isinstance(value, str) and len(value) > 50:
                print(f"      • {key}: {value[:50]}...")
            else:
                print(f"      • {key}: {value}")
else:
    print("\n⚠️  No conversations found to inspect actual data")

print("\n" + "=" * 60)

🔍 Inspecting Conversation model...
✅ Conversation Model Fields:
   Fields: ['id', 'title', 'raw_text', 'source', 'word_count', 'created_at', 'status']

📋 Field Details:
      • id: <class 'int'>
      • title: typing.Optional[str]
      • raw_text: <class 'str'>
      • source: <class 'str'>
      • word_count: <class 'int'>
      • created_at: <class 'datetime.datetime'>
      • status: <class 'str'>

✅ Sample Conversation Data:
   Type: <class 'database.models.Conversation'>
   Available attributes: ['id', 'title', 'raw_text', 'source', 'word_count', 'created_at', 'status']

   Sample Values:
      • id: 28
      • title: Audio: blog_record_(purevitalize).wav
      • raw_text: My name is Michael Chen. I'm the Chief Technology ...
      • source: transcribed
      • word_count: 298
      • created_at: 2025-11-28 09:02:37
      • status: pending



In [43]:
# Cell: Safe Helper Functions (Updated)
def get_conv_attribute(conv, attr_name, default='Unknown'):
    """Safely get attribute from conversation (works with dict or Pydantic model)"""
    if isinstance(conv, dict):
        return conv.get(attr_name, default)
    else:
        # For Pydantic models, use model_dump() to get all fields as dict
        try:
            data = conv.model_dump()
            return data.get(attr_name, default)
        except AttributeError:
            # Fallback to getattr
            return getattr(conv, attr_name, default)

def get_conv_filename(conv):
    """Get filename from conversation, trying multiple possible attribute names"""
    # Try common attribute names for audio file
    possible_names = [
        'audio_file_path',  # Most common
        'file_path',        # Alternative
        'filename',         # Simple name
        'audio_file',       # Without path
        'source_file',      # Source reference
        'audio_path',       # Path variant
        'file_name'         # Snake case variant
    ]
    
    for attr in possible_names:
        filename = get_conv_attribute(conv, attr, None)
        if filename:
            return filename
    
    # If nothing found, create a descriptive name from ID and timestamp
    conv_id = get_conv_attribute(conv, 'id', 'unknown')
    created = get_conv_attribute(conv, 'created_at', '')
    
    if created:
        return f"Conversation_{conv_id}_{str(created)[:10]}"
    else:
        return f"Conversation_{conv_id}"

def get_idea_attribute(idea, attr_name, default=0):
    """Safely get attribute from blog idea (works with dict or Pydantic model)"""
    if isinstance(idea, dict):
        return idea.get(attr_name, default)
    else:
        try:
            data = idea.model_dump()
            return data.get(attr_name, default)
        except AttributeError:
            return getattr(idea, attr_name, default)

print("✅ Safe helper functions ready (Pydantic v2.11 compatible)")

✅ Safe helper functions ready (Pydantic v2.11 compatible)


In [44]:
# Run the inspection to see what fields are actually available
print("🔍 Running Conversation model inspection...")
print("\n" + "="*60)

from database.models import Conversation

# Show model fields
print("📋 Conversation Model Fields (from class):")
for field_name in Conversation.model_fields.keys():
    print(f"   • {field_name}")

# Get sample data
conversations = db.get_all_conversations()
if conversations:
    sample = conversations[0]
    print(f"\n✅ Sample conversation available")
    print(f"   ID: {get_conv_attribute(sample, 'id')}")
    print(f"   Filename: {get_conv_filename(sample)}")
    
    # Show all available data
    if isinstance(sample, dict):
        print(f"\n   All fields in sample:")
        for key in sample.keys():
            print(f"      • {key}")
    else:
        data = sample.model_dump()
        print(f"\n   All fields in sample:")
        for key in data.keys():
            print(f"      • {key}")
else:
    print("\n⚠️  No conversations in database yet")

print("="*60)

🔍 Running Conversation model inspection...

📋 Conversation Model Fields (from class):
   • id
   • title
   • raw_text
   • source
   • word_count
   • created_at
   • status

✅ Sample conversation available
   ID: 28
   Filename: Conversation_28_2025-11-28

   All fields in sample:
      • id
      • title
      • raw_text
      • source
      • word_count
      • created_at
      • status


In [49]:
# Make sure helper functions are loaded first
print("✅ Helper functions loaded:", 
      'get_conv_attribute' in dir() and 
      'get_conv_filename' in dir() and 
      'get_idea_attribute' in dir())

# Now test
print("\n🧪 Testing FIXED functions...")
print("\n1. List Conversations:")
list_conversations()

✅ Helper functions loaded: True

🧪 Testing FIXED functions...

1. List Conversations:

💬 ALL CONVERSATIONS IN DATABASE

📁 ID: 28
   🎙️  File: Conversation_28_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 46.4/70
   📅 Created: 2025-11-28 09:02:37
   🔗 View: show_ideas_dashboard(28)

📁 ID: 27
   🎙️  File: Conversation_27_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 35.0/70
   📅 Created: 2025-11-28 08:27:30
   🔗 View: show_ideas_dashboard(27)

📁 ID: 26
   🎙️  File: Conversation_26_2025-11-27
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 35.0/70
   📅 Created: 2025-11-27 09:11:49
   🔗 View: show_ideas_dashboard(26)

📁 ID: 25
   🎙️  File: Conversation_25_2025-11-26
   📝 Transcript: 0 characters
   💡 Ideas: 0 | No ideas yet
   📅 Created: 2025-11-26 09:11:52
   🔗 View: show_ideas_dashboard(25)

📁 ID: 24
   🎙️  File: Conversation_24_2025-11-21
   📝 Transcript: 0 characters
   💡 Ideas: 0 | No ideas yet
   📅 Created: 2025-11-21 09:58:56
   🔗

In [48]:
# Test the fixed functions
print("🧪 Testing fixed functions...")
print("\n1. List Conversations:")
list_conversations()

print("\n2. Explore and Visualize:")
explore_and_visualize()

🧪 Testing fixed functions...

1. List Conversations:

💬 ALL CONVERSATIONS IN DATABASE

📁 ID: 28
   🎙️  File: Conversation_28_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 46.4/70
   📅 Created: 2025-11-28 09:02:37
   🔗 View: show_ideas_dashboard(28)

📁 ID: 27
   🎙️  File: Conversation_27_2025-11-28
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 35.0/70
   📅 Created: 2025-11-28 08:27:30
   🔗 View: show_ideas_dashboard(27)

📁 ID: 26
   🎙️  File: Conversation_26_2025-11-27
   📝 Transcript: 0 characters
   💡 Ideas: 5 | Avg Score: 35.0/70
   📅 Created: 2025-11-27 09:11:49
   🔗 View: show_ideas_dashboard(26)

📁 ID: 25
   🎙️  File: Conversation_25_2025-11-26
   📝 Transcript: 0 characters
   💡 Ideas: 0 | No ideas yet
   📅 Created: 2025-11-26 09:11:52
   🔗 View: show_ideas_dashboard(25)

📁 ID: 24
   🎙️  File: Conversation_24_2025-11-21
   📝 Transcript: 0 characters
   💡 Ideas: 0 | No ideas yet
   📅 Created: 2025-11-21 09:58:56
   🔗 View: show_ideas_dashboard(24)



In [50]:
# Cell: Quick inspection
from database.models import Conversation

print("📋 Conversation model fields:")
print(list(Conversation.model_fields.keys()))

# Get a sample
conversations = db.get_all_conversations()
if conversations:
    sample = conversations[0]
    print(f"\n✅ Sample conversation data:")
    data = sample.model_dump() if hasattr(sample, 'model_dump') else sample
    for key, value in data.items():
        if isinstance(value, str) and len(value) > 50:
            print(f"   • {key}: {value[:50]}...")
        else:
            print(f"   • {key}: {value}")

📋 Conversation model fields:
['id', 'title', 'raw_text', 'source', 'word_count', 'created_at', 'status']

✅ Sample conversation data:
   • id: 28
   • title: Audio: blog_record_(purevitalize).wav
   • raw_text: My name is Michael Chen. I'm the Chief Technology ...
   • source: transcribed
   • word_count: 298
   • created_at: 2025-11-28 09:02:37
   • status: pending
